<a href="https://colab.research.google.com/github/Samiimasmoudii/Deep-Learning-Course/blob/main/Manual_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chargement et préparation des données CIFAR-10
Le dataset CIFAR-10 contient 60 000 images couleur de dimensions 32x32 réparties en 10 classes.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import random
from scipy.stats import uniform


# Charger le dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalisation des données (les valeurs des pixels sont mises entre 0 et 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Encodage des étiquettes (one-hot encoding)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Diviser les données d'entraînement en un ensemble d'entraînement et de validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Fonction pour créer un CNN avec des hyperparamètres configurables
def create_cnn(num_filters=32, kernel_size=3, learning_rate=0.001, dropout_rate=0.5):
    model = Sequential([
        Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------------------------------------------------------------------------------------------------------------------
# 1. GRID SEARCH : Recherche exhaustive sur un ensemble fixe d'hyperparamètres
# ----------------------------------------------------------------------------------------------------------------------

# Définir les hyperparamètres à explorer
grid_params = {
    'num_filters': [32, 64],        # Nombre de filtres
    'kernel_size': [3, 5],          # Taille des kernels
    'learning_rate': [0.001, 0.01], # Taux d'apprentissage
    'dropout_rate': [0.3, 0.5],     # Taux de dropout
    'epochs': [5, 10]               # Nombre d'époques
}

# Exploration de toutes les combinaisons
best_accuracy = 0
best_params = None

for num_filters in grid_params['num_filters']:
    for kernel_size in grid_params['kernel_size']:
        for learning_rate in grid_params['learning_rate']:
            for dropout_rate in grid_params['dropout_rate']:
                for epochs in grid_params['epochs']:
                    print(f"Testing configuration: filters={num_filters}, kernel_size={kernel_size}, lr={learning_rate}, "
                          f"dropout={dropout_rate}, epochs={epochs}")

                    model = create_cnn(num_filters, kernel_size, learning_rate, dropout_rate)
                    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=32, verbose=0)

                    val_accuracy = max(history.history['val_accuracy'])
                    if val_accuracy > best_accuracy:
                        best_accuracy = val_accuracy
                        best_params = {
                            'num_filters': num_filters,
                            'kernel_size': kernel_size,
                            'learning_rate': learning_rate,
                            'dropout_rate': dropout_rate,
                            'epochs': epochs
                        }

print(f"Best Grid Search Parameters: {best_params} with accuracy: {best_accuracy}")



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Testing configuration: filters=32, kernel_size=3, lr=0.001, dropout=0.3, epochs=5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing configuration: filters=32, kernel_size=3, lr=0.001, dropout=0.3, epochs=10
Testing configuration: filters=32, kernel_size=3, lr=0.001, dropout=0.5, epochs=5
Testing configuration: filters=32, kernel_size=3, lr=0.001, dropout=0.5, epochs=10
Testing configuration: filters=32, kernel_size=3, lr=0.01, dropout=0.3, epochs=5
Testing configuration: filters=32, kernel_size=3, lr=0.01, dropout=0.3, epochs=10
Testing configuration: filters=32, kernel_size=3, lr=0.01, dropout=0.5, epochs=5
Testing configuration: filters=32, kernel_size=3, lr=0.01, dropout=0.5, epochs=10
Testing configuration: filters=32, kernel_size=5, lr=0.001, dropout=0.3, epochs=5
Testing configuration: filters=32, kernel_size=5, lr=0.001, dropout=0.3, epochs=10
Testing configuration: filters=32, kernel_size=5, lr=0.001, dropout=0.5, epochs=5
Testing configuration: filters=32, kernel_size=5, lr=0.001, dropout=0.5, epochs=10
Testing configuration: filters=32, kernel_size=5, lr=0.01, dropout=0.3, epochs=5
Testing configu

In [ ]:
# ----------------------------------------------------------------------------------------------------------------------
# 2. RANDOM SEARCH : Recherche aléatoire sur un espace d'hyperparamètres défini
# ----------------------------------------------------------------------------------------------------------------------

# Définir l'espace des hyperparamètres
random_params = {
    'num_filters': [16, 32, 64, 128],
    'kernel_size': [3, 5, 7],
    'learning_rate': uniform(0.0001, 0.01).rvs(10),  # Tirages aléatoires
    'dropout_rate': uniform(0.2, 0.5).rvs(10),       # Tirages aléatoires
    'epochs': [5, 10, 15]
}

num_trials = 10
best_accuracy_random = 0
best_params_random = None

for i in range(num_trials):
    # Tirer des valeurs aléatoires
    num_filters = random.choice(random_params['num_filters'])
    kernel_size = random.choice(random_params['kernel_size'])
    learning_rate = random.choice(random_params['learning_rate'])
    dropout_rate = random.choice(random_params['dropout_rate'])
    epochs = random.choice(random_params['epochs'])

    print(f"Testing random configuration: filters={num_filters}, kernel_size={kernel_size}, lr={learning_rate}, "
          f"dropout={dropout_rate}, epochs={epochs}")

    model = create_cnn(num_filters, kernel_size, learning_rate, dropout_rate)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=32, verbose=0)

    val_accuracy = max(history.history['val_accuracy'])
    if val_accuracy > best_accuracy_random:
        best_accuracy_random = val_accuracy
        best_params_random = {
            'num_filters': num_filters,
            'kernel_size': kernel_size,
            'learning_rate': learning_rate,
            'dropout_rate': dropout_rate,
            'epochs': epochs
        }

print(f"Best Random Search Parameters: {best_params_random} with accuracy: {best_accuracy_random}")



Testing random configuration: filters=16, kernel_size=3, lr=0.003668908770406429, dropout=0.2649199115291976, epochs=5
Testing random configuration: filters=32, kernel_size=7, lr=0.007860516713686242, dropout=0.4323308230259747, epochs=5
Testing random configuration: filters=16, kernel_size=3, lr=0.005028749992460761, dropout=0.3036293926242864, epochs=15
Testing random configuration: filters=64, kernel_size=7, lr=0.005028749992460761, dropout=0.6282445426721383, epochs=15
